In [1]:
your_name = 'yunwei'

In [ ]:
import cpuinfo
import dataclasses
import numpy
import platform
import primme
import psutil
import qutip
import sys
import scipy
import scqubits
import time
import timeit
import tqdm

from threadpoolctl import threadpool_info, threadpool_limits

scqubits.settings.PROGRESSBAR_DISABLED = True

def make_numcpus_list():
    all_cpus = qutip.utilities.available_cpu_count()
    if all_cpus == 1:
        return [1]
    if all_cpus == 2:
        return [1, 2]
    if all_cpus <= 4:
        return [1, 2, 4]
    return numpy.linspace(1, all_cpus - 2, 4, dtype=int).tolist()



BM_ITEMS = [
    "test_id",
    "matmode",
    "solver",
    "num_cpus",
    "num_threads",
    "which",
    "sigma",
    "ncv",
    "check_finite",
    "other",
    "time"
]


SOLVERS = {
    "scipy_eigsh": scipy.sparse.linalg.eigsh,
    "primme_eigsh": primme.eigsh
}


TEST_CONFIG_SPARSE = {
    "ZeroPi": {
        "matmode": "sparse",
        "solver":   ["scipy_eigsh", "primme_eigsh"],
        "num_cpus": [1], # ***********************  do sequential testing for now, no multiprocessing on scqubits side
        "num_threads": make_numcpus_list(),
        "which":  ['SA', 'SM', 'LA', 'LM'],
        "ncv": [None, "plus5", "plus10"],
        "evals_count": [6, 10, 20]
    },
    "Cos2PhiQubit": {
        "matmode": "sparse",
        "solver": ["scipy_eigsh", "primme_eigsh"],
        "num_cpus": [1], # *********************** do sequential testing for now, no multiprocessing on scqubits side
        "num_threads": make_numcpus_list(),
        "which":  ['SA', 'SM', 'LA', 'LM'],
        "ncv": [None, "plus5", "plus10"],
        "evals_count": [6, 10, 20]
    },
    # Dropping FullZeroPi for now, takes a long time. Will have to think about reducing overall benchmarks runtime to be an acceptable ask.
    #
    # "FullZeroPi": {
    #     "matmode": "sparse",
    #     "solver": ["scipy_eigsh", "primme_eigsh"],
    #     "num_cpus": [1], # ***********************make_numcpus_list(),
    #     "num_threads": make_numcpus_list(),
    #     "which": ['SA', 'SM', 'LA', 'LM'],
    #     "ncv": [None, "plus5", "plus10"],
    #     "evals_count": [6, 10, 20]
    # }
}

fluxvals = numpy.linspace(0, 0.5, 2)   # 12)



PAYLOAD = {}



@dataclasses.dataclass()
class BenchmarkData:
    test_id: str
    num_cpus: int
    num_threads: int
    which: str
    sigma: str
    ncv: int
    k: int
    eval_0: float
    time: float




def acquire_metadata() -> None:
    global PAYLOAD
    cpu = cpuinfo.get_cpu_info()
    PAYLOAD["platform"] = platform.platform()
    PAYLOAD["machine"] = platform.machine()
    if "M1" in cpu['brand_raw']:
        PAYLOAD["cpuinfo"] = f"{cpu['brand_raw']}"
    else:
        PAYLOAD["cpuinfo"] = f"{cpu['brand_raw']} {cpu['hz_advertised_friendly']}, act. {cpu['hz_actual_friendly']}"
    PAYLOAD["python_version"] = "%d.%d.%d" % sys.version_info[0:3]
    PAYLOAD["cpu_count"] = "%s" % qutip.utilities.available_cpu_count()
    PAYLOAD["cpu_load_percent"] = psutil.cpu_percent(4)
    PAYLOAD["ram_load_percent"] = psutil.virtual_memory()[2]
    PAYLOAD["ram_load_gb"] = psutil.virtual_memory()[3]/1000000000
    PAYLOAD["python_compiler"] = platform.python_compiler()
    PAYLOAD["scqubits_version"] = scqubits.version.version
    PAYLOAD["numpy_version"] = numpy.version.version
    PAYLOAD["qutip_version"] = qutip.__version__
    try:
        import Cython
        cython_ver = Cython.__version__
    except ImportError:
        cython_ver = 'None'
    PAYLOAD["cython_version"] = cython_ver

    thp_info = threadpool_info()
    for idx, infodict in enumerate(thp_info):
        PAYLOAD[f"threadpool.{idx}.user_api"] = thp_info[idx]['user_api']
        PAYLOAD[f"threadpool.{idx}.internal_api"] = thp_info[idx]['internal_api']
        PAYLOAD[f"threadpool.{idx}.prefix"] = thp_info[idx]['prefix']
        PAYLOAD[f"threadpool.{idx}.version"] = thp_info[idx]['version']
        PAYLOAD[f"threadpool.{idx}.threading_layer"] = thp_info[idx].get('threading_layer')
        PAYLOAD[f"threadpool.{idx}.num_threads"] = thp_info[idx]['num_threads']

    PAYLOAD["blas"] = "%s" % qutip.utilities._blas_info()
    PAYLOAD["openmp"] =  "%s" % str(qutip.settings.has_openmp)
    PAYLOAD["intel_mkl_ext"] = "%s" % str(qutip.settings.has_mkl)

    from numpy.core._multiarray_umath import (__cpu_features__, __cpu_baseline__, __cpu_dispatch__)
    for name, info_dict in numpy.__config__.__dict__.items():
        if name[0] == "_" or type(info_dict) is not type({}): continue
        if not info_dict:
            continue
        for k,v in info_dict.items():
            if "_dir" in k:
                continue
            v = str(v)
            if k == "sources" and len(v) > 200:
                v = v[:60] + " ...\n... " + v[-60:]
            PAYLOAD[f"numpy.{name}.{k}"] = v

    features_found, features_not_found = [], []
    for feature in __cpu_dispatch__:
        if __cpu_features__[feature]:
            features_found.append(feature)
        else:
            features_not_found.append(feature)
    PAYLOAD[f"numpy.simdext.baseline"] = ','.join(__cpu_baseline__)
    PAYLOAD[f"numpy.simdext.found"] = ','.join(features_found)
    PAYLOAD[f"numpy.simdext.notfound"] = ','.join(features_not_found)





In [74]:
SPARSE_BENCHMARKS = []  # List of BenchmarkData instances


def timeit_and_returnval(func, *args, number=1, **kwargs):
    """Used to get return value from benchmarked function back."""
    output_container = []
    def wrapper():
        output_container.append(func(*args, **kwargs))
    timer = timeit.Timer(wrapper)
    delta = timer.timeit(number)
    return delta, output_container.pop()


def sparse_qubit_benchmarks(verbose=True):
    total_count = sum([len(qbt_dict["solver"]) * len(qbt_dict["num_cpus"]) * len(qbt_dict["num_threads"]) * len(qbt_dict["which"]) * len(qbt_dict["evals_count"])  * len(qbt_dict["ncv"]) for qbt, qbt_dict in TEST_CONFIG_SPARSE.items()])

    with tqdm.notebook.tqdm(total=total_count) as pbar:
        for qubit_name in TEST_CONFIG_SPARSE.keys():
            qbt_class = getattr(scqubits, qubit_name)
            qbt = qbt_class.create()

            bm_idx = 0

            for solver_name in TEST_CONFIG_SPARSE[qubit_name]["solver"]:
                solver = SOLVERS[solver_name]
                for ncpu in TEST_CONFIG_SPARSE[qubit_name]["num_cpus"]:
                    for nthreads in TEST_CONFIG_SPARSE[qubit_name]["num_threads"]:
                        with threadpool_limits(limits=nthreads, user_api='blas'):
                            for which in TEST_CONFIG_SPARSE[qubit_name]["which"]:
                                for k in TEST_CONFIG_SPARSE[qubit_name]["evals_count"]:
                                    for ncv in TEST_CONFIG_SPARSE[qubit_name]["ncv"]:

                                        if ncv == "plus5":
                                            ncv = max(2*k + 1, 20) + 5
                                        elif ncv == "plus10":
                                            ncv = max(2*k + 1, 20) + 10

                                        if which in ["LA", "LM"]:
                                            if "primme" in solver_name:
                                                which = "S" + which[1]
                                            sigma = 0.0
                                        else:
                                            sigma = None

                                        def custom_esys(matrix, *args, **kwargs):
                                            evals, evecs = solver(
                                                matrix, 
                                                k=k,
                                                which=which,
                                                sigma=sigma,
                                                ncv=ncv
                                            )
                                            return evals, evecs

                                        qbt.esys_method = custom_esys
                                        
                                        try:
                                            time_needed, spec = timeit_and_returnval(qbt.get_spectrum_vs_paramvals, "flux", fluxvals, get_eigenstates=True, num_cpus=ncpu)
                                        except Exception as exc:
                                            time_needed = -1
    
                                        
                                        
                                        bm = BenchmarkData(
                                            test_id=f"{qubit_name} {solver_name} {bm_idx}",
                                            num_cpus=ncpu,
                                            num_threads=nthreads,
                                            which=which,
                                            sigma=sigma,
                                            ncv=ncv,
                                            k=k,
                                            eval_0=spec.energy_table[0][0],
                                            time=time_needed
                                        )
                                        if verbose:
                                            print(bm)
                                            
                                        SPARSE_BENCHMARKS.append(bm)
                                        bm_idx += 1
                                        pbar.update(1)




In [75]:
acquire_metadata()
sparse_qubit_benchmarks()

Output()

BenchmarkData(test_id='ZeroPi scipy_eigsh 0', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=None, k=6, eval_0=16.993839848490385, time=1.3537833230002434)
BenchmarkData(test_id='ZeroPi scipy_eigsh 1', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=25, k=6, eval_0=16.993839848490385, time=1.2983918719983194)
BenchmarkData(test_id='ZeroPi scipy_eigsh 2', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=30, k=6, eval_0=16.993839848490385, time=1.2772790320013883)
BenchmarkData(test_id='ZeroPi scipy_eigsh 3', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=None, k=10, eval_0=16.993839848490385, time=1.2794534819986438)
BenchmarkData(test_id='ZeroPi scipy_eigsh 4', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=26, k=10, eval_0=16.993839848490385, time=1.2825891770044109)
BenchmarkData(test_id='ZeroPi scipy_eigsh 5', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=31, k=10, eval_0=16.993839848490385, time=1.3621334780036705)
BenchmarkData(test_id

BenchmarkData(test_id='ZeroPi scipy_eigsh 51', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=None, k=20, eval_0=16.993839848490385, time=1.6255765649984824)
BenchmarkData(test_id='ZeroPi scipy_eigsh 52', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=46, k=20, eval_0=16.993839848490385, time=1.3714290030038683)
BenchmarkData(test_id='ZeroPi scipy_eigsh 53', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=51, k=20, eval_0=16.993839848490385, time=1.3850096829992253)
BenchmarkData(test_id='ZeroPi scipy_eigsh 54', num_cpus=1, num_threads=2, which='LA', sigma=0.0, ncv=None, k=6, eval_0=16.993839848490385, time=1.4020853799956967)
BenchmarkData(test_id='ZeroPi scipy_eigsh 55', num_cpus=1, num_threads=2, which='LA', sigma=0.0, ncv=25, k=6, eval_0=16.993839848490385, time=1.366690169001231)
BenchmarkData(test_id='ZeroPi scipy_eigsh 56', num_cpus=1, num_threads=2, which='LA', sigma=0.0, ncv=30, k=6, eval_0=16.993839848490385, time=1.3733927619978203)
BenchmarkData(test_

BenchmarkData(test_id='ZeroPi scipy_eigsh 102', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=None, k=10, eval_0=16.993839848490385, time=1.3469906699974672)
BenchmarkData(test_id='ZeroPi scipy_eigsh 103', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=26, k=10, eval_0=16.993839848490385, time=1.3816741049959091)
BenchmarkData(test_id='ZeroPi scipy_eigsh 104', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=31, k=10, eval_0=16.993839848490385, time=1.3645406720024766)
BenchmarkData(test_id='ZeroPi scipy_eigsh 105', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=None, k=20, eval_0=16.993839848490385, time=1.393347042001551)
BenchmarkData(test_id='ZeroPi scipy_eigsh 106', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=46, k=20, eval_0=16.993839848490385, time=1.3503289069994935)
BenchmarkData(test_id='ZeroPi scipy_eigsh 107', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=51, k=20, eval_0=16.993839848490385, time=1.3384726500007673)
BenchmarkData

BenchmarkData(test_id='ZeroPi primme_eigsh 152', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=51, k=20, eval_0=16.993839848490385, time=1.4683570659981342)
BenchmarkData(test_id='ZeroPi primme_eigsh 153', num_cpus=1, num_threads=1, which='SM', sigma=None, ncv=None, k=6, eval_0=16.993839848490385, time=1.4114488329942105)
BenchmarkData(test_id='ZeroPi primme_eigsh 154', num_cpus=1, num_threads=1, which='SM', sigma=None, ncv=25, k=6, eval_0=16.993839848490385, time=1.345630046002043)
BenchmarkData(test_id='ZeroPi primme_eigsh 155', num_cpus=1, num_threads=1, which='SM', sigma=None, ncv=30, k=6, eval_0=16.993839848490385, time=1.3307356759978575)
BenchmarkData(test_id='ZeroPi primme_eigsh 156', num_cpus=1, num_threads=1, which='SM', sigma=None, ncv=None, k=10, eval_0=16.993839848490385, time=1.3377505469979951)
BenchmarkData(test_id='ZeroPi primme_eigsh 157', num_cpus=1, num_threads=1, which='SM', sigma=None, ncv=26, k=10, eval_0=16.993839848490385, time=1.3382049169958918)
Benc

...

BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 49', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=26, k=10, eval_0=12.477406428829164, time=0.6053941220015986)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 50', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=31, k=10, eval_0=12.477406428829164, time=0.5985254839979461)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 51', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=None, k=20, eval_0=12.477406428829164, time=0.6145673319988418)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 52', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=46, k=20, eval_0=12.477406428829164, time=0.6229472939958214)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 53', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=51, k=20, eval_0=12.477406428829164, time=0.6157542990040383)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 54', num_cpus=1, num_threads=2, which='LA', sigma=0.0, ncv=None, k=6, eval_0=12.477406428829164, tim

BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 98', num_cpus=1, num_threads=4, which='LA', sigma=0.0, ncv=51, k=20, eval_0=12.477406428829164, time=0.615648084996792)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 99', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=None, k=6, eval_0=12.477406428829164, time=0.6202292580055655)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 100', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=25, k=6, eval_0=12.477406428829164, time=0.6229750269994838)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 101', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=30, k=6, eval_0=12.477406428829164, time=0.6306707580006332)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 102', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=None, k=10, eval_0=12.477406428829164, time=0.6110401659971103)
BenchmarkData(test_id='Cos2PhiQubit scipy_eigsh 103', num_cpus=1, num_threads=4, which='LM', sigma=0.0, ncv=26, k=10, eval_0=12.477406428829164, time=0.

BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 147', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=None, k=10, eval_0=12.477406428829164, time=0.6289029170002323)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 148', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=26, k=10, eval_0=12.477406428829164, time=0.6184777460002806)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 149', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=31, k=10, eval_0=12.477406428829164, time=0.6232053729981999)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 150', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=None, k=20, eval_0=12.477406428829164, time=0.6350342969963094)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 151', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=46, k=20, eval_0=12.477406428829164, time=0.6024225130022387)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 152', num_cpus=1, num_threads=1, which='SA', sigma=None, ncv=51, k=20, eval_0=12.477406

BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 195', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=None, k=20, eval_0=12.477406428829164, time=0.6052645970048616)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 196', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=46, k=20, eval_0=12.477406428829164, time=0.6019071339978836)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 197', num_cpus=1, num_threads=2, which='SM', sigma=None, ncv=51, k=20, eval_0=12.477406428829164, time=0.6036291959972004)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 198', num_cpus=1, num_threads=2, which='SA', sigma=0.0, ncv=None, k=6, eval_0=12.477406428829164, time=0.6035553830006393)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 199', num_cpus=1, num_threads=2, which='SA', sigma=None, ncv=25, k=6, eval_0=12.477406428829164, time=0.6329937589980545)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 200', num_cpus=1, num_threads=2, which='SA', sigma=None, ncv=30, k=6, eval_0=12.4774064288

BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 243', num_cpus=1, num_threads=4, which='SM', sigma=0.0, ncv=None, k=6, eval_0=12.477406428829164, time=0.6084704040040378)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 244', num_cpus=1, num_threads=4, which='SM', sigma=None, ncv=25, k=6, eval_0=12.477406428829164, time=0.6054785619999166)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 245', num_cpus=1, num_threads=4, which='SM', sigma=None, ncv=30, k=6, eval_0=12.477406428829164, time=0.604831919001299)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 246', num_cpus=1, num_threads=4, which='SM', sigma=None, ncv=None, k=10, eval_0=12.477406428829164, time=0.6030239969986724)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 247', num_cpus=1, num_threads=4, which='SM', sigma=None, ncv=26, k=10, eval_0=12.477406428829164, time=0.6057165739985066)
BenchmarkData(test_id='Cos2PhiQubit primme_eigsh 248', num_cpus=1, num_threads=4, which='SM', sigma=None, ncv=31, k=10, eval_0=12.47740642882

In [80]:
import pickle
with open(your_name+'_benchmarks_sparse.pkl', 'wb') as f:  # open a file
    pickle.dump(SPARSE_BENCHMARKS, f) # serialize the list

In [81]:
import pandas
df = pandas.DataFrame(SPARSE_BENCHMARKS)

In [82]:
for key in PAYLOAD:
#     print(PAYLOAD[key])
    PAYLOAD[key]=[PAYLOAD[key]]



In [83]:
df_pl=pandas.DataFrame.from_dict(PAYLOAD)
# Repeat DataFrame 'df_b' to match the number of rows in DataFrame 'a'
b_repeated = pandas.concat([df_pl] * len(df), ignore_index=True)

# Concatenate 'a' and the repeated 'b' horizontally
result = pandas.concat([df, b_repeated], axis=1)
result.to_excel(your_name+'.xlsx')